<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ תהליך הסבת שכבת קדסטר מבוסס קואורדינטות לבנק"ל מודרני 🗺️ 

In [ ]:
print('9. Coordinates Based Cadastre Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  9.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
טבלת מספרי גושים של קמ"ק

In [ ]:
CBC_list = pd.read_excel(CNFG.Local + 'CoordBasedCadstreBlocksList.xlsx', usecols = ['BlockNumber','SubBlockNumber'])

<h1 style="text-align:right;font-size:100%">
שכבת גושי קדסטר

In [ ]:
blocks_columns = ['BlockNumber','SubBlockNumber', 'BlockUniqueID', 'geometry']

Blocks = daskgpd.read_file(CNFG.Results, layer = 'Blocks', columns = blocks_columns, npartitions = 16).compute()

Blocks[['BlockNumber','SubBlockNumber', 'BlockUniqueID']] = Blocks[['BlockNumber','SubBlockNumber', 'BlockUniqueID']].astype(int)

del blocks_columns

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  9.b Cleaning data')

In [ ]:
print('  9.c Data conversion')

<h1 style="text-align:right;font-size:100%">
מיזוג מידע

In [ ]:
merge = Blocks.merge(CBC_list, on = ['BlockNumber','SubBlockNumber'], how = 'right')
merge = merge.drop_duplicates(keep = 'first')
merge = merge[~merge['geometry'].isna()]

del[CBC_list, Blocks]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  9.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                        ("BlockUniqueID", int),
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),
                        ("Status", int),
                        #("PlanName", str),
                        #("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])

CBC_Blocks = gpd.GeoDataFrame(data     = np.empty(0, dtype = new_columns),
                              geometry = 'geometry',
                              crs      = 2039)

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
CBC_Blocks['BlockUniqueID'] = merge['BlockUniqueID'].astype('Int64')

CBC_Blocks['ProcessType'] = 5

CBC_Blocks['MapYear'] = 2013

CBC_Blocks['ProcessName'] = 'קמ"ק ' + merge['BlockNumber'].astype(str) + '/' + merge['SubBlockNumber'].astype(str)

CBC_Blocks['GeodeticNetwork'] = 3

CBC_Blocks['Status'] = 13 #מאושר

#CBC_Blocks['PlanName'] = None

#CBC_Blocks['SurveyorLicenseID'] = None

CBC_Blocks['DataSource'] = 5 #CAD

CBC_Blocks['geometry'] = merge['geometry']

CBC_Blocks['CPBUniqueID'] = range(5000000, 5000000+len(CBC_Blocks))

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  9.e Exporting results to product folder')

In [ ]:
json_dir = CNFG.JsonPath + 'CoordinatesBasedCadastreBorders.geojson'

CBC_Blocks.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + "CoordinatesBasedCadastreBorders - Save as Feature Class.bat"]);